In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import os   
from torch.utils.tensorboard import SummaryWriter
from data import OralCancerDataset, RotationTransform
from utils import save_checkpoint, load_checkpoint, latest_checkpoint_path
from process import train_epoch
from models import *

batch_size = 192
learning_rate = 1e-04
betas = (0.9, 0.999)
num_workers = 4
pin_memory = True
drop_path_rate = 0.2
epochs = 360
save_freq = 10


output_csv = 'outputs/submission_all_data_nxt_25_14__.csv'
path_to_csv = 'cancer-classification-challenge-2024/train.csv'
path_to_train_images = 'cancer-classification-challenge-2024/train'
path_to_test_images = 'cancer-classification-challenge-2024/test'
model_dir = 'checkpoints/'

transform = transforms.Compose([
    RotationTransform(angles=[0, -90, 90, 180]),
    transforms.Resize((128, 128)), 
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    transforms.Resize((128, 128)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_dataset = OralCancerDataset(path_to_csv=path_to_csv, path_to_image=path_to_train_images, transform=transform, full_dataset=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = get_convnext_tiny_AvgPolling(drop_path_rate=drop_path_rate).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=betas)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer,start_factor=1,end_factor=0.1,total_iters=400)
writer = SummaryWriter()


for epoch in range(epochs):
    train_loss, trian_acc, lr = train_epoch(model, optimizer, scheduler, loss_fn, train_loader, device)
    
    writer.add_scalar("Loss/train", train_loss, epoch)
    writer.add_scalar("Accuracy/train", trian_acc, epoch)
    writer.add_scalar("HyperParameters/LearningRate", lr , epoch)
    print(f"Epoch: {epoch:3d}/{epochs:3d} Train loss: {train_loss:6f} Train acc: {trian_acc:6f}, LR: {lr:6f}")

    if epoch > 1 and epoch%save_freq == 0:
        save_checkpoint(model, optimizer, scheduler, epoch, model_dir, max_checkpoints = 10)
        print(f"Model saved at epoch {epoch}")


image_files = [f for f in os.listdir(path_to_test_images) if f.endswith('.jpg')]
results = []

model.eval()
with torch.no_grad():
    for i, image_file in enumerate(image_files):
        image_path = os.path.join(path_to_test_images, image_file)
        image = Image.open(image_path).convert('RGB')
        image = transform_val(image).unsqueeze(0).to(device)
        outputs = model(image)
        probabilities = F.softmax(outputs, dim=1)
        prob_class_1 = probabilities[0][1].item()  
        if (i % 100 == 0):
            print(f'Processed {i}/{len(image_files)} images')

        results.append((image_file, prob_class_1))

df = pd.DataFrame(results, columns=['Name', 'Diagnosis'])
df.to_csv(output_csv, index=False)

Epoch:   0/360 Train loss: 0.360541 Train acc: 0.830695, LR: 0.000100
Epoch:   1/360 Train loss: 0.242886 Train acc: 0.893217, LR: 0.000100
Epoch:   2/360 Train loss: 0.183230 Train acc: 0.922004, LR: 0.000099
Epoch:   3/360 Train loss: 0.151282 Train acc: 0.938059, LR: 0.000099
Epoch:   4/360 Train loss: 0.125179 Train acc: 0.950007, LR: 0.000099


KeyboardInterrupt: 